In [1]:
# Importing Libraries

from py_vollib.black_scholes import black_scholes as bs
from py_vollib.black_scholes.greeks.analytical import vega
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from IPython.display import HTML, Image # For GIF
from matplotlib import rc
rc('animation', html='jshtml')

In [2]:
# Defining the Implied Volatity Function

#Compute the implied volatility of a European Option and add use the following variables

""""
S0: initial stock price
K:  strike price
T:  maturity
r:  risk-free rate
market_price: option price in market
tol: user choosen tolerance
"""

# We will also add a flag for 'c' for call and 'p' for put

def implied_vol(S0, K, T, r, market_price, flag='c', tol=0.00001):
  
    max_iter = 200 #max number of iterations
    vol_old = 0.30 #initial guess

    # We will calcualte the Black Scholes price, Vega and Initial Volatility

    for k in range(max_iter):
        bs_price = bs(flag, S0, K, T, r, vol_old)
        Cprime =  vega(flag, S0, K, T, r, vol_old)*100 # We will multiply by 100 because of this specific module output
        C = bs_price - market_price
        
        
        vol_new = vol_old - C/Cprime
        bs_new = bs(flag, S0, K, T, r, vol_new)
        
        if (abs(vol_old - vol_new) < tol or abs(bs_new - market_price) < tol):
            break

        vol_old = vol_new
    implied_vol = vol_old

    return implied_vol

In [3]:
# Add the values for an Exampls

S0, K, T, r = 30, 28, 0.2, 0.025

market_price = 5.00
implied_vol_est = implied_vol(S0, K, T, r, market_price, flag='c')


In [4]:
# Print the Implied Volatility
 
print("Implied Volatility is : ", round(implied_vol_est,2)*100, "%")

Implied Volatility is :  75.0 %
